# Python and pandas for deep learning
> Using Python and pandas for working with your files

In this notebook, we examine how one can work with files, particularly with pandas to get the data in a desired format for usage with deep learning packages.  Here, we'll examine how to use Google Colab with Google Drive mounted so that you'll be able to access your data files.  Note that there are many ways to achieve this; one method is described below!

# Mounting with Google Colab

The instructions for mounting Google Drive are concisely described in the 0 notebook.  For more options about uploading and interacting with Google Colab, see their IO notebook here: https://colab.research.google.com/notebooks/io.ipynb .

Here, the approach we will take is as follows.  First, we'll mount Google Drive via https://colab.research.google.com .  So we'll all have access to the same files, we'll go ahead and clone this repository in Google Drive.  This will give us access to the repository and file contents programmatically.  Note that when you first mount, you may have to enter some credentials and provide some approvals.  The steps are outlined here.

## 1.  Mount Google Drive
Note that you can also use the mount button in the sidebar.  Follow all steps to provide approval and access.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2.  Clone the repo
Again, we're cloning the repo mostly just so we'll have the desired files that we want and you can see how to do it using Google Drive.  Here, we'll simulate this by cloning within your Google Drive.  You can see this with the `cd` (Change Directory) command below.

Again, keep in mind that if your data already exists somewhere on your drive, you wouldn't necessarily have to do this unless you wanted to access the repo on your drive.  You would use the filepaths directly as shown in a following step.

In [ ]:
%cd drive/MyDrive
!git clone https://github.com/vanderbilt-data-science/deep-learning-intensive.git
%cd deep-learning-intensive

## 3. Install and load the required modules
We need to pip install transformers.  We'll do this below.  The rest are already available through Google Colab.

In [ ]:
!pip install transformers

In [ ]:
#import data science packages
import pandas as pd
import numpy as np
import seaborn as sns

#import file helper packages
import glob

#dl imports
from transformers import pipeline

# Example scenario
You're working with the text of number of magazine articles, written by a number of authors, whose names are not included in the text.  The magazine articles have unique IDs, which are given by their filenames.  

You also have an accompanying CSV file with details about all of the authors, including their name, age, years of employment as a journalist, college major, and IDs corresponding to the articles that they've written.  

You're interested in learning whether their college major affects the sentiments of their articles.  The only data that you have access to is gathered on a particular day over a set of journalists and articles.  You've decided that the first crack you'll take about understanding this relationship is to separate the journalists (or articles) into groups according to their college major, and count the sentiments for all the corresponding articles for each group.

## Access data
The first thing you'll need to do is access your data and be able to load it for processing with Python.  We'll use `glob` for this.  Note that we've changed into the repo directory, meaning that the filepath to `workshop-files` is `workshop-files` with no other additional paths necessary.

In [ ]:
filenames = glob.glob('workshop-files/*.txt')
filenames

### Get file contents
We're going to be using the requests package to read from a remote file (the repo on GitHub).  You can use the `requests.get(insert-url-path-here.txt)` function to get the contents of a remote url.  Let's use a list comprehension to implement this functionality.  We'll do this together!

In [ ]:
#read file contents
file_contents = []
for file in filenames:
    with open(file, 'r') as f:
        file_contents.append(f.read())

In [ ]:
print('Length of file contents list:', len(file_contents))
file_contents[0][:100]

### Read tabular data
Here, we'll read the tabular data stored in the csv about the authors.

In [ ]:
#concatenate file path
author_df = pd.read_csv('workshop-files/author_data.csv')
author_df.head()

### Creating dataframes from lists/dictionaries
Now, we'll make a dataframe of the files and concatenate them to the df_list.  Note that the split character below `/` may need to be changed to `\\` on windows machines.

In [ ]:
#first, create a dataframe from your filename list and your text
tinfo_df = pd.DataFrame({'filename':[fname.split('/')[-1] for fname in filenames], 'text':file_contents})
tinfo_df['article_id'] = tinfo_df['filename'].apply(lambda x: int(x.split('.')[0]))
tinfo_df.head()

### Putting data together: joining data
Now, we can join the data together.  There are many different types of joins, but essentially a join will look for one or more key, match the key, and then bind the columns of both dataframes together where the key matches.  We'll do a simple inner join here.

In [ ]:
print("Text info columns:", tinfo_df.columns.tolist())
print("Author columns:", author_df.columns.tolist())

In [ ]:
#join tables
full_df = pd.merge(author_df, tinfo_df, on='article_id')

#print some info
display(full_df.head())
print(full_df.shape)

## Modeling Results
Now that we have all of our data together and are able to extract the text of any desired subset, let's do some modeling!  Let's use a sentiment analysis pipeline to get the results.
### In-class exercise
Using the text classification pipeline, pass in either the list of texts or iterate through the texts to get infererences on each of the article texts.  Fill in the cell below with the required code.

In [ ]:
#get modeling results
#sentiment_classifier = add your code here and uncomment
#sent_results = add your code here and uncomment

In [ ]:
#create dataframe from list of dictionaries
sent_df = pd.DataFrame(sent_results)
sent_df

In [ ]:
#concatenate dataframes
final_df = pd.concat([author_df, sent_df], axis=1)
final_df.head()

## Now, let's actually answer our question!
Can we see a difference between the sentiments of articles based on college major?

In [ ]:
final_df.groupby(['college major'])['label'].value_counts().unstack().transpose().plot(kind='bar');

## What if we wanted to compute an aggregate score?
Looking at negative and positive is a bit hard.  Maybe we could just calculate a score reflecting whether overall, the group tends to have positive reviews.  We can calculate this as `no_positive_reviews - no_negative_reviews`.

In [ ]:
negpos_df = final_df.groupby(['college major'])['label'].value_counts().unstack().fillna(0)
negpos_df

In [ ]:
negpos_df['overall_positivity'] = negpos_df['POSITIVE'] - negpos_df['NEGATIVE']
negpos_df['overall_positivity'].transpose().plot(kind='bar');

# What we've covered
We've covered a lot of ground today!  We've discussed several things:
* Reading in text files via Google Colab and locally via the glob package
* Example of HuggingFace sentiment-analysis model inference using demo function
* Converting outputs to dataframes

# Homework assignment
Upload your data to Google Drive if you can, and make sure that you can access it programmatically!  We'll be working on your own data for the next few classes.  If something prevents you from uploading data to cloud storage, make sure you follow the instructions in the `0-Installations` notebook to install Anaconda locally.  Make sure you install HuggingFace transformers and all required Pytorch (or TensorFlow) packages. 